In [31]:
import os
import sqlite3
import pandas as pd
from datetime import datetime, timedelta
import time
import calendar

In [38]:
# Create empty database if not exists

dbLoc = "data.sqlite3"
if not os.path.exists(dbLoc):
    with sqlite3.connect(dbLoc) as db:
        # Datatypes from https://earthquake.usgs.gov/data/comcat/data-eventterms.php
        db.execute("""
            CREATE TABLE earthquakes (
                time TEXT,
                latitude NUMERIC,
                longitude NUMERIC,
                depth NUMERIC,
                mag NUMERIC,
                magType NUMERIC,
                nst INTEGER,
                gap NUMERIC,
                dmin NUMERIC,
                rms INTEGER,
                net TEXT,
                id TEXT,
                updated INTEGER,
                place TEXT,
                type TEXT,
                horizontalError NUMERIC,
                depthError NUMERIC,
                magError NUMERIC,
                magNst INTEGER,
                status TEXT,
                locationSource TEXT,
                magSource TEXT,
                scrapeTime INTEGER
            );
        """)

In [37]:
# Pull data from USGS
# 
# As we're pulling a lot of data, we'll do it one request at a time to be nice

with sqlite3.connect(dbLoc) as db:
    nSaved = 0
    minMagnitude = 2.5
    startTime = datetime(2010, 1, 1, 0, 0)
    endTime = datetime(2019, 7, 15, 0, 0)

    lastStartTime=startTime
    while lastStartTime < endTime:
        scrapeEndTime = min(lastStartTime + timedelta(days=20), endTime)
        print(f"Scraping {lastStartTime} to {scrapeEndTime}")

        nScraped = 0
        while True:
            # https://earthquake.usgs.gov/fdsnws/event/1/
            options=[
                f"starttime={lastStartTime.strftime('%Y-%m-%d%%20%H:%M:%S')}",
                f"endtime={scrapeEndTime.strftime('%Y-%m-%d%%20%H:%M:%S')}",
                f"minmagnitude={minMagnitude}",
                f"orderby=time",
                f"limit=20000",
                f"offset={nScraped + 1}"
            ]
            df = pd.read_csv(f"https://earthquake.usgs.gov/fdsnws/event/1/query.csv?{'&'.join(options)}")
            df['scrapeTime'] = int(time.time())
            nScraped += len(df)

            df.to_sql("earthquakes", db, if_exists='append', index=False)

            # Throttle ourselves
            time.sleep(0.3)

            # Move on the next time range if complete
            if len(df) < 20000:
                break
            offset += len(df)

        nSaved += nScraped
        print(f"    Scraped {nScraped} ({nSaved} total)")
        lastStartTime = scrapeEndTime

Scraping 2010-01-01 00:00:00 to 2010-01-21 00:00:00
    Scraped 968 (968 total)
Scraping 2010-01-21 00:00:00 to 2010-02-10 00:00:00
    Scraped 802 (1770 total)
Scraping 2010-02-10 00:00:00 to 2010-03-02 00:00:00
    Scraped 1480 (3250 total)
Scraping 2010-03-02 00:00:00 to 2010-03-22 00:00:00
    Scraped 1514 (4764 total)
Scraping 2010-03-22 00:00:00 to 2010-04-11 00:00:00
    Scraped 2558 (7322 total)
Scraping 2010-04-11 00:00:00 to 2010-05-01 00:00:00
    Scraped 1627 (8949 total)
Scraping 2010-05-01 00:00:00 to 2010-05-21 00:00:00
    Scraped 1160 (10109 total)
Scraping 2010-05-21 00:00:00 to 2010-06-10 00:00:00
    Scraped 1003 (11112 total)
Scraping 2010-06-10 00:00:00 to 2010-06-30 00:00:00
    Scraped 1164 (12276 total)
Scraping 2010-06-30 00:00:00 to 2010-07-20 00:00:00
    Scraped 1725 (14001 total)
Scraping 2010-07-20 00:00:00 to 2010-08-09 00:00:00
    Scraped 2125 (16126 total)
Scraping 2010-08-09 00:00:00 to 2010-08-29 00:00:00
    Scraped 1284 (17410 total)
Scraping 2010

    Scraped 1277 (129123 total)
Scraping 2015-06-24 00:00:00 to 2015-07-14 00:00:00
    Scraped 1311 (130434 total)
Scraping 2015-07-14 00:00:00 to 2015-08-03 00:00:00
    Scraped 1858 (132292 total)
Scraping 2015-08-03 00:00:00 to 2015-08-23 00:00:00
    Scraped 1515 (133807 total)
Scraping 2015-08-23 00:00:00 to 2015-09-12 00:00:00
    Scraped 1586 (135393 total)
Scraping 2015-09-12 00:00:00 to 2015-10-02 00:00:00
    Scraped 2056 (137449 total)
Scraping 2015-10-02 00:00:00 to 2015-10-22 00:00:00
    Scraped 1367 (138816 total)
Scraping 2015-10-22 00:00:00 to 2015-11-11 00:00:00
    Scraped 1552 (140368 total)
Scraping 2015-11-11 00:00:00 to 2015-12-01 00:00:00
    Scraped 1517 (141885 total)
Scraping 2015-12-01 00:00:00 to 2015-12-21 00:00:00
    Scraped 1382 (143267 total)
Scraping 2015-12-21 00:00:00 to 2016-01-10 00:00:00
    Scraped 1418 (144685 total)
Scraping 2016-01-10 00:00:00 to 2016-01-30 00:00:00
    Scraped 1412 (146097 total)
Scraping 2016-01-30 00:00:00 to 2016-02-19 0